In [0]:
import torch 
import torch.nn as nn
import torchvision
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
import random;
import math;

In [0]:
num_epochs = 5;
batch_size = 100;
learning_rate = 0.001;

In [13]:
dataset = torchvision.datasets.FashionMNIST('/content', train=True, download=True)

Processing...
Done!


In [6]:
train_dataset = FashionMNISTDataset(csv_file='fashionmnist/fashion-mnist_train.csv');

FileNotFoundError: ignored

In [8]:
import os
os.getcwd()

'/content'